Counterfactual explanations systematically remove some items from the user's
previous interactions and then call on the recommendation system again to check if the
item they want to provide an explanation for is removed from the user's suggestions. If
the item is successfully removed, then the set of items responsible for that alteration is
the explanation provided to the user. This would result in an explanation of the form: ''If
you had not liked item A, then item B would not have been suggested,'' where A is a set
of items removed from the user's feedback, and B is the item the user wanted an
explanation for.

In this part of the project, the goal is to produce counterfactual explanations for a group
of users. Specifically, design (10 points) and implement (10 points) a method that
generates counterfactual explanations that adhere to some characteristics to be more
in tune with group recommendations. For example, an explanation that only consists of
items interacted by a single user is undesirable since it would single out that user to the
rest of the group. Ideally, we would like explanations that consist of items most users
have interacted with to make the explanation fairer. In this context, it means that no
user should have changed their preferences for the group, but the group as a whole is
responsible for the suggestions provided by the system. Prepare also a short
presentation (about 5 slides) to show how your method works (5 points).

In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from sklearn.metrics import jaccard_score
from scipy.spatial.distance import pdist, squareform
from itertools import combinations
from datetime import datetime

In [2]:
links = pd.read_csv('ml-latest-small/links.csv')
movies = pd.read_csv('ml-latest-small/movies.csv')
ratings = pd.read_csv('ml-latest-small/ratings.csv')
tags = pd.read_csv('ml-latest-small/tags.csv')

print("Links Dataset:")
display(links.head())

print("\nMovies Dataset:")
display(movies.head())

print("\nRatings Dataset:")
display(ratings.head())

print("\nTags Dataset:")
display(tags.head())

rating_count = ratings.shape[0]
print(f"\nTotal number of ratings: {rating_count}")

Links Dataset:


,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0



Movies Dataset:


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy



Ratings Dataset:


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931



Tags Dataset:


,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200



Total number of ratings: 100836


In [3]:
# Create user-item matrix for cosine similarity
user_item_matrix = ratings.pivot(index='userId', columns='movieId', values='rating').fillna(0)

# Calculate cosine similarity between users
similarity_matrix = cosine_similarity(user_item_matrix)
similarity_df = pd.DataFrame(similarity_matrix, index=user_item_matrix.index, columns=user_item_matrix.index)

group_users = [6, 7, 8, 1, 2]

# Satisfaction scores for the users (initially all unsatisfied)
satisfaction_scores = {user: 0.0 for user in group_users}

# Keep track of movies already recommended for each user and group
user_history = {user: set() for user in group_users}
group_history = set()

# Function to get movie names
def get_movie_title(movie_id):
    title = movies[movies['movieId'] == movie_id]['title']
    return title.values[0] if not title.empty else "Unknown"

# Function to apply fairness-aware weights
def capped_weights(weights, cap=2.0):
    return {user: min(weight, cap) for user, weight in weights.items()}

# Function to recommend for a user based on similarity
def recommend_for_user(user_id, user_item_matrix, similarity_df, num_recommendations=5):
    similar_users = similarity_df[user_id].nlargest(5)  # Limit to top 5 similar users for efficiency
    user_rated_movies = user_item_matrix.loc[user_id]
    watched_movies = set(user_rated_movies[user_rated_movies > 0].index)
    scores = np.zeros(user_item_matrix.shape[1])
    
    for similar_user, similarity_score in similar_users.items():
        if similar_user == user_id:
            continue
        scores += user_item_matrix.loc[similar_user].values * similarity_score
    
    scores = pd.Series(scores, index=user_item_matrix.columns)
    scores = scores[~scores.index.isin(watched_movies | user_history[user_id] | group_history)]

    # Genre-matching adjustment
    user_ratings = ratings[ratings['userId'] == user_id]
    high_rated_movies = user_ratings[user_ratings['rating'] >= 4.0]['movieId']
    high_rated_genres = movies[movies['movieId'].isin(high_rated_movies)]['genres']
    user_genres = set([genre for genres in high_rated_genres for genre in genres.split('|')])
    
    for movie_id in scores.index:
        movie_genres = set(movies[movies['movieId'] == movie_id]['genres'].values[0].split('|'))
        genre_match = len(user_genres & movie_genres)
        scores[movie_id] += genre_match * 0.1

    recommended_movies = scores.sort_values(ascending=False).head(num_recommendations)
    user_history[user_id].update(recommended_movies.index)
    return [(movie_id, score) for movie_id, score in recommended_movies.items()]

# Aggregation with fairness
def aggregate_group_recommendations(user_recommendations, weights, max_recommendations=10):
    weights = capped_weights(weights, cap=2.0)  # Apply fairness cap
    scores = defaultdict(float)
    for user, recommendations in user_recommendations.items():
        weight = weights[user]
        for movie_id, score in recommendations:
            scores[movie_id] += score * weight
    
    scores = {movie: score for movie, score in scores.items() if movie not in group_history}
    sorted_recommendations = sorted(scores.items(), key=lambda x: -x[1])
    top_recommendations = [movie for movie, _ in sorted_recommendations[:max_recommendations]]
    group_history.update(top_recommendations)
    return top_recommendations

# # WNCF explanation generation (Note: The usefulness of WNCF explanations might be limited in this context, consider if it adds meaningful insights)
def generate_wncf_explanation(group_users, user_item_matrix, similarity_df, item, k=5):
    explanation = []
    
    # Step 1: Check if item is in the recommendation set
    if item not in user_item_matrix.columns:
        explanation.append("I: The item was not in the recommendation set.")
        return explanation

    # Step 2: Check if item has no ratings
    item_ratings = ratings[ratings['movieId'] == item]['rating']
    if item_ratings.empty:
        explanation.append("S: The item has no ratings.")
        return explanation

    # Step 3: Check if there is a tie in the relevance score
    for user_id in group_users:
        similar_users = similarity_df[user_id].nlargest(5)  # Limit to top 5 similar users for efficiency
        user_rated_items = user_item_matrix.loc[user_id]
        watched_items = set(user_rated_items[user_rated_items > 0].index)
        
        scores = np.zeros(user_item_matrix.shape[1])
        for similar_user, similarity_score in similar_users.items():
            if similar_user == user_id:
                continue
            scores += user_item_matrix.loc[similar_user].values * similarity_score
        
        scores = pd.Series(scores, index=user_item_matrix.columns)
        if item in scores.index and scores[item] in scores.values:
            tied_item = scores.idxmax()
            explanation.append(f"Tie: There was a tie in relevance score with item '{get_movie_title(tied_item)}', which indicates multiple items had similar relevance to user preferences.")
            break

    # Step 5: Check if at least one peer has rated the item
    peers_rated = [peer for peer in group_users if peer != user_id and item in user_item_matrix.columns and user_item_matrix.at[peer, item] > 0]
    if peers_rated:
        avg_rating = np.mean([user_item_matrix.at[peer, item] for peer in peers_rated])
        explanation.append(f"Peers: {len(peers_rated)} peers have rated the item with an average score of {avg_rating:.1f}. This suggests that the item was popular among multiple group members, contributing to its recommendation.")
    else:
        explanation.append("Peers: None of the peers have rated this item, indicating a potential gap in group interest.")

    # Step 6: Check if less than numPI similar peers have rated the item
    numPI = 3  # Threshold for similar peers
    rated_by_similar_peers = len(peers_rated)
    if rated_by_similar_peers < numPI:
        explanation.append("numPI: Less than the threshold number of similar peers have rated the item, suggesting limited relevance among users with similar preferences.")
    
    return explanation

# Function to get movies commonly watched by group members
def get_common_watched_movies(group_users, user_item_matrix):
    # Create a matrix of movies watched by all group members
    group_watched = user_item_matrix.loc[group_users].astype(bool).sum(axis=0)
    # Select only movies watched by at least two group members
    common_watched_movies = group_watched[group_watched >= 2].nlargest(5).index  # Limit to top 5 most watched movies
    return common_watched_movies

# Counterfactual explanation generation for group
def generate_counterfactual_group_explanation(group_users, target_item, user_item_matrix, similarity_df):
    common_watched_movies = get_common_watched_movies(group_users, user_item_matrix)
    explanation = []

    # Print target item information for better context
    print(f"\nTarget Item for Explanation: {get_movie_title(target_item)} ({target_item})")

    # Iterate over common watched movies
    for movie in common_watched_movies:
        # Zero out the relevant columns directly instead of creating a full copy
        original_values = user_item_matrix.loc[group_users, movie].copy()
        user_item_matrix.loc[group_users, movie] = 0

        # Recalculate group recommendations
        new_group_recommendations = aggregate_group_recommendations(
            {user: recommend_for_user(user, user_item_matrix, similarity_df) for user in group_users},
            {user: 1.0 for user in group_users}
        )

        # Restore the original values
        user_item_matrix.loc[group_users, movie] = original_values

        # Check if target item is still in group recommendations
        if target_item not in new_group_recommendations:
            users_interacted = [user for user in group_users if user_item_matrix.at[user, movie] > 0]
            if len(users_interacted) == 1:
                explanation.append(f"If user {users_interacted[0]} had not liked '{get_movie_title(movie)}', then '{get_movie_title(target_item)}' would not have been suggested. This shows that '{get_movie_title(movie)}' had a significant influence on recommending '{get_movie_title(target_item)}'.")
            else:
                explanation.append(f"If users {', '.join(map(str, users_interacted))} had not liked '{get_movie_title(movie)}', then '{get_movie_title(target_item)}' would not have been suggested. This shows that '{get_movie_title(movie)}' had a significant influence on recommending '{get_movie_title(target_item)}'.")

    if not explanation:
        return f"No common item removal was found to affect the recommendation of '{get_movie_title(target_item)}'. This indicates that '{target_item}' was recommended due to diverse preferences across the group."
    else:
        return "\n".join(explanation)

# Example usage
group_recommendations = aggregate_group_recommendations(
    {user: recommend_for_user(user, user_item_matrix, similarity_df) for user in group_users},
    {user: 1.0 for user in group_users}
)
print("\nGroup Recommendations:")
for movie_id in group_recommendations:
    print(f"- {get_movie_title(movie_id)}")

# Generate a WNCF explanation for a specific item
item_to_explain = group_recommendations[0]
wncf_explanation = generate_wncf_explanation(group_users, user_item_matrix, similarity_df, item_to_explain)
print("\nWNCF Explanation:")
print("\n".join(wncf_explanation))

# Generate a counterfactual explanation for the group
group_counterfactual_explanation = generate_counterfactual_group_explanation(group_users, item_to_explain, user_item_matrix, similarity_df)
print("\nCounterfactual Group Explanation:")
print(group_counterfactual_explanation)


# Function to get movie details
def get_movie_details(movie_id):
    movie_row = movies[movies['movieId'] == movie_id]
    if not movie_row.empty:
        title = movie_row['title'].values[0]
        genres = movie_row['genres'].values[0]
        return title, genres
    return "Unknown", "Unknown"

# Genre-level reasoning
def genre_preferences_reasoning(group_users, ratings, movies):
    """
    Calculate group preferences for genres based on ratings.
    """
    group_ratings = ratings[ratings['userId'].isin(group_users)]
    genre_scores = defaultdict(float)
    genre_counts = defaultdict(int)

    for _, row in group_ratings.iterrows():
        movie_id = row['movieId']
        rating = row['rating']
        _, genres = get_movie_details(movie_id)

        for genre in genres.split('|'):
            genre_scores[genre] += rating
            genre_counts[genre] += 1

    # Calculate average ratings for each genre
    avg_genre_scores = {genre: genre_scores[genre] / genre_counts[genre] for genre in genre_scores}
    return avg_genre_scores

# Dependency reasoning
def dependency_reasoning(movie_id, similarity_df, top_k=5):
    """
    Explain why a movie was recommended by analyzing its dependencies (similar movies).
    """
    reasoning = []
    similar_items = similarity_df.loc[movie_id].sort_values(ascending=False).head(top_k)
    for similar_item, similarity_score in similar_items.items():
        title, _ = get_movie_details(similar_item)
        reasoning.append(
            f"Movie '{title}' is similar to '{get_movie_details(movie_id)[0]}' (Similarity: {similarity_score:.2f})."
        )
    return reasoning

# Peer-based collaborative filtering reasoning
def peer_reasoning(group_users, movie_id, user_item_matrix, similarity_df):
    """
    Explain recommendations based on peer ratings and similarities.
    """
    reasoning = []
    for user in group_users:
        similar_users = similarity_df[user].sort_values(ascending=False).head(5)  # Top 5 similar users
        for peer, similarity in similar_users.items():
            if peer == user:
                continue
            peer_rating = user_item_matrix.at[peer, movie_id]
            if peer_rating > 0:
                reasoning.append(
                    f"Peer {peer} rated '{get_movie_details(movie_id)[0]}' {peer_rating} (Similarity: {similarity:.2f})."
                )
    return reasoning

# Granularity in explanations
def granularity_explanation(movie_id, genre_preferences, common_watched_movies):
    """
    Provide explanations at both atomic and group levels.
    """
    explanations = []

    # Genre-level insights
    _, genres = get_movie_details(movie_id)
    for genre in genres.split('|'):
        if genre in genre_preferences:
            explanations.append(f"Genre '{genre}' is preferred by the group (Avg rating: {genre_preferences[genre]:.2f}).")

    # Group-level insights
    if movie_id in common_watched_movies:
        explanations.append(f"'{get_movie_details(movie_id)[0]}' was commonly watched by group members.")

    return explanations

# Enhanced recommendation explanations
def generate_advanced_explanations(group_users, user_item_matrix, similarity_df, ratings, movies, target_item):
    """
    Generate a comprehensive explanation for group recommendations.
    """
    # Calculate genre preferences and commonly watched movies
    genre_preferences = genre_preferences_reasoning(group_users, ratings, movies)
    common_watched_movies = get_common_watched_movies(group_users, user_item_matrix)

    # Build explanations
    explanations = []

    # Peer reasoning
    explanations.extend(peer_reasoning(group_users, target_item, user_item_matrix, similarity_df))

    # Dependency reasoning
    explanations.extend(dependency_reasoning(target_item, similarity_df))

    # Granularity explanation
    explanations.extend(granularity_explanation(target_item, genre_preferences, common_watched_movies))

    # Model-specific tie-breaking explanation
    tie_break_explanation = generate_wncf_explanation(group_users, user_item_matrix, similarity_df, target_item)
    explanations.extend(tie_break_explanation)

    return explanations

# Example Usage
item_to_explain = group_recommendations[0]
advanced_explanations = generate_advanced_explanations(
    group_users, user_item_matrix, similarity_df, ratings, movies, item_to_explain
)

print("\nAdvanced Explanations:")
for explanation in advanced_explanations:
    print(f"- {explanation}")



Group Recommendations:
- Terminator 2: Judgment Day (1991)
- Die Hard: With a Vengeance (1995)
- Fight Club (1999)
- Stargate (1994)
- Clueless (1995)
- Crimson Tide (1995)
- Cliffhanger (1993)
- Nightmare Before Christmas, The (1993)
- Pulp Fiction (1994)
- Aliens (1986)

WNCF Explanation:
Tie: There was a tie in relevance score with item 'Forrest Gump (1994)', which indicates multiple items had similar relevance to user preferences.
Peers: 1 peers have rated the item with an average score of 2.5. This suggests that the item was popular among multiple group members, contributing to its recommendation.
numPI: Less than the threshold number of similar peers have rated the item, suggesting limited relevance among users with similar preferences.

Target Item for Explanation: Terminator 2: Judgment Day (1991) (589)

Counterfactual Group Explanation:
If users 6, 7, 8, 1 had not liked 'Usual Suspects, The (1995)', then 'Terminator 2: Judgment Day (1991)' would not have been suggested. This 